In [40]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import mnist
from time import time
import os
print(tf.__version__)

2.13.0


Checkpoint function

In [41]:
def load(model, checkpoint_dir):
    print(" [*] Reading checkpoints...")

    ckpt = tf.train.get_checkpoint_state(checkpoint_dir)
    if ckpt :
        ckpt_name = os.path.basename(ckpt.model_checkpoint_path)
        checkpoint = tf.train.Checkpoint(dnn=model)
        checkpoint.restore(save_path=os.path.join(checkpoint_dir, ckpt_name))
        counter = int(ckpt_name.split('-')[1])
        print(" [*] Success to read {}".format(ckpt_name))
        return True, counter
    else:
        print(" [*] Failed to find a checkpoint")
        return False, 0

def check_folder(dir):
    if not os.path.exists(dir):
        os.makedirs(dir)
    return dir

Data load & pre-processing function

In [42]:
def load_mnist() :
    (train_data, train_labels), (test_data, test_labels) = mnist.load_data()
    train_data = np.expand_dims(train_data, axis=-1) # [N, 28, 28] -> [N, 28, 28, 1]
    test_data = np.expand_dims(test_data, axis=-1) # [N, 28, 28] -> [N, 28, 28, 1]

    train_data, test_data = normalize(train_data, test_data) # 0~255 를 0~1로

    train_labels = to_categorical(train_labels, 10) # [N,] -> [N, 10]
    test_labels = to_categorical(test_labels, 10) # [N,] -> [N, 10]

    return train_data, train_labels, test_data, test_labels

def normalize(train_data, test_data):
    train_data = train_data.astype(np.float32) / 255.0
    test_data = test_data.astype(np.float32) / 255.0

    return train_data, test_data

np.expand_dims(train_data, axis=-1) 에서는 차원을 [N, 28, 28] 에서 [N, 28, 28, 1] 으로 확장시켜주었으며,
np.expand_dims(train_data, axis= 0) 에서는 차원을 [N, 28, 28] 에서 [1, N, 28, 28] 으로 확장시켜주었다.
이는 tensorflow가 데이터를 [batch size, height, width, channel]의 형식으로 받기 때문이다

to categorical 함수를 이용하는 이유는 one-hot encoding을 위해서인데 이를 통해 train label을 [7]이 아닌 [0,0,0,0,0,0,0,1,0,0]로 표현한다.

Performance function

In [43]:
def loss_fn(model, images, labels):
    logits = model(images, training=True)    
    loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y_pred=logits, y_true=labels, 
                                                                   from_logits=True))
    return loss

def accuracy_fn(model, images, labels):
    logits = model(images, training=False)
    prediction = tf.equal(tf.argmax(logits, -1), tf.argmax(labels, -1)) #argmax 을 통해 가장 큰 logits, labels를 갖는 값의 위치 표시
    accuracy = tf.reduce_mean(tf.cast(prediction, tf.float32)) #tf.cast를 통해 True False의 값을 1,0 으로 변환해준다
    return accuracy

def grad(model, images, labels):
    with tf.GradientTape() as tape:
        loss = loss_fn(model, images, labels)
    return tape.gradient(loss, model.variables)

Model function

In [44]:
def flatten() :
    return tf.keras.layers.Flatten()

def dense(label_dim, weight_init) :
    return tf.keras.layers.Dense(units=label_dim, use_bias=True, kernel_initializer=weight_init)

def sigmoid() :
    return tf.keras.layers.Activation(tf.keras.activations.sigmoid)

Create model in class version

In [45]:
class create_model_class(tf.keras.Model):
    def __init__(self, label_dim): #여기서 label_dim = 10
        super(create_model_class, self).__init__()
        weight_init = tf.keras.initializers.RandomNormal()

        self.model = tf.keras.Sequential()
        self.model.add(flatten()) #[N, 28, 28, 1] 을 [N, 784] 로 바꿔준다

        for i in range(2):
            self.model.add(dense(256, weight_init)) #[N, 784]에서 [N, 256]
            self.model.add(sigmoid())

        self.model.add(dense(label_dim, weight_init)) #[N, 256]에서 [N, 10]

    def call(self, x, training=None, mask=None):
        x = self.model(x)

        return x

Create model in function version

In [46]:
def create_model_function(label_dim) :
    weight_init = tf.keras.initializers.RandomNormal()

    model = tf.keras.Sequential()
    model.add(flatten())

    for i in range(2) :
        model.add(dense(256, weight_init))
        model.add(sigmoid())

    model.add(dense(label_dim, weight_init))

    return model

Define data & Hyper-parameter

In [47]:
""" dataset """
train_x, train_y, test_x, test_y = load_mnist()

""" parameters """
learning_rate = 0.001
batch_size = 128

training_epochs = 1
training_iterations = len(train_x) // batch_size

label_dim = 10

train_flag = True

""" Graph Input using Dataset API """
train_dataset = tf.data.Dataset.from_tensor_slices((train_x, train_y)).\
    shuffle(buffer_size=100000).\
    prefetch(buffer_size=batch_size).\
    batch(batch_size, drop_remainder=True)

test_dataset = tf.data.Dataset.from_tensor_slices((test_x, test_y)).\
    shuffle(buffer_size=100000).\
    prefetch(buffer_size=len(test_x)).\
    batch(len(test_x))

데이터셋 API를 이용해서 각각의 데이터와 이미지와 라벨들을 네트워크에 넣는 과정
여기서 train data는 60000개, test data는 10000개이므로 한번에 다 넣어주게 되면 메모리에 부담이 된다.
따라서 barch size 만큰 나눠서 넣어준다.
또한 buffer size 는 input으로 들어가는 data의 갯수보다 큰 값인 100000으로 설정했다

Define model & optimizer & writer

In [48]:
""" Model """
network = create_model_function(label_dim)

""" Training """
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

""" Writer """
checkpoint_dir = 'checkpoints'
logs_dir = 'C:/Users/johnk/logs/'

model_dir = 'nn_softmax'

checkpoint_dir = os.path.join(checkpoint_dir, model_dir)
check_folder(checkpoint_dir)
checkpoint_prefix = os.path.join(checkpoint_dir, model_dir)
logs_dir = os.path.join(logs_dir, model_dir)

Restore checkpoint & start train or test phrase

In [49]:
if True :

    checkpoint = tf.train.Checkpoint(dnn=network) #학습을 하다가 끊겼을 때 다시 재학습을 하는 경우 학습하면서 변경된 weight들을
                                                  #불러내는 역할을 한다.
    # create writer for tensorboard
    summary_writer = tf.summary.create_file_writer("C:/Users/johnk/logs/")
    start_time = time()

    # restore check-point if it exits
    could_load, checkpoint_counter = load(network, checkpoint_dir)    

    if could_load:
        start_epoch = (int)(checkpoint_counter / training_iterations)        
        counter = checkpoint_counter        
        print(" [*] Load SUCCESS")
    else:
        start_epoch = 0
        start_iteration = 0
        counter = 0
        print(" [!] Load failed...")
    
    # train phase
    with summary_writer.as_default():  # for tensorboard
        for epoch in range(start_epoch, training_epochs):
            for idx, (train_input, train_label) in enumerate(train_dataset):            
                grads = grad(network, train_input, train_label)
                optimizer.apply_gradients(grads_and_vars=zip(grads, network.variables))

                train_loss = loss_fn(network, train_input, train_label)
                train_accuracy = accuracy_fn(network, train_input, train_label)
                
                for test_input, test_label in test_dataset:                
                    test_accuracy = accuracy_fn(network, test_input, test_label)

                tf.summary.scalar(name='train_loss', data=train_loss, step=counter)
                tf.summary.scalar(name='train_accuracy', data=train_accuracy, step=counter)
                tf.summary.scalar(name='test_accuracy', data=test_accuracy, step=counter)

                print(
                    "Epoch: [%2d] [%5d/%5d] time: %4.4f, train_loss: %.8f, train_accuracy: %.4f, test_Accuracy: %.4f" \
                    % (epoch, idx, training_iterations, time() - start_time, train_loss, train_accuracy,
                       test_accuracy))
                counter += 1                
        #checkpoint.save(file_prefix=checkpoint_prefix + '-{}'.format(counter))
        
# test phase      
else :
    _, _ = load(network, checkpoint_dir)
    for test_input, test_label in test_dataset:    
        test_accuracy = accuracy_fn(network, test_input, test_label)

    print("test_Accuracy: %.4f" % (test_accuracy))

 [*] Reading checkpoints...
 [*] Failed to find a checkpoint
 [!] Load failed...
Epoch: [ 0] [    0/  468] time: 0.7201, train_loss: 2.31330013, train_accuracy: 0.1094, test_Accuracy: 0.0958
Epoch: [ 0] [    1/  468] time: 0.8986, train_loss: 2.26805806, train_accuracy: 0.1250, test_Accuracy: 0.1135
Epoch: [ 0] [    2/  468] time: 1.0971, train_loss: 2.27682137, train_accuracy: 0.0781, test_Accuracy: 0.1161
Epoch: [ 0] [    3/  468] time: 1.2955, train_loss: 2.25574398, train_accuracy: 0.2266, test_Accuracy: 0.1654
Epoch: [ 0] [    4/  468] time: 1.5169, train_loss: 2.28234291, train_accuracy: 0.1094, test_Accuracy: 0.0980
Epoch: [ 0] [    5/  468] time: 1.7872, train_loss: 2.29316545, train_accuracy: 0.0938, test_Accuracy: 0.0980
Epoch: [ 0] [    6/  468] time: 2.0545, train_loss: 2.30237317, train_accuracy: 0.0781, test_Accuracy: 0.1007
Epoch: [ 0] [    7/  468] time: 2.2709, train_loss: 2.23543930, train_accuracy: 0.2422, test_Accuracy: 0.1883
Epoch: [ 0] [    8/  468] time: 2.4734,

Epoch: [ 0] [   74/  468] time: 17.2419, train_loss: 0.99088520, train_accuracy: 0.7891, test_Accuracy: 0.7690
Epoch: [ 0] [   75/  468] time: 17.4264, train_loss: 0.92678523, train_accuracy: 0.8047, test_Accuracy: 0.7744
Epoch: [ 0] [   76/  468] time: 17.6428, train_loss: 0.96133125, train_accuracy: 0.7734, test_Accuracy: 0.7862
Epoch: [ 0] [   77/  468] time: 17.8861, train_loss: 1.05130458, train_accuracy: 0.7188, test_Accuracy: 0.7994
Epoch: [ 0] [   78/  468] time: 18.1135, train_loss: 0.86712319, train_accuracy: 0.8281, test_Accuracy: 0.8095
Epoch: [ 0] [   79/  468] time: 18.3439, train_loss: 0.91739702, train_accuracy: 0.8203, test_Accuracy: 0.8189
Epoch: [ 0] [   80/  468] time: 18.5833, train_loss: 0.87171900, train_accuracy: 0.8125, test_Accuracy: 0.8226
Epoch: [ 0] [   81/  468] time: 18.8426, train_loss: 0.93106943, train_accuracy: 0.7969, test_Accuracy: 0.8260
Epoch: [ 0] [   82/  468] time: 19.0839, train_loss: 0.85520113, train_accuracy: 0.8672, test_Accuracy: 0.8200
E

Epoch: [ 0] [  149/  468] time: 35.2138, train_loss: 0.48658079, train_accuracy: 0.8906, test_Accuracy: 0.8829
Epoch: [ 0] [  150/  468] time: 35.5329, train_loss: 0.47746369, train_accuracy: 0.8750, test_Accuracy: 0.8812
Epoch: [ 0] [  151/  468] time: 35.8002, train_loss: 0.58473712, train_accuracy: 0.8516, test_Accuracy: 0.8803
Epoch: [ 0] [  152/  468] time: 36.0435, train_loss: 0.56630999, train_accuracy: 0.8516, test_Accuracy: 0.8812
Epoch: [ 0] [  153/  468] time: 36.3348, train_loss: 0.59633517, train_accuracy: 0.8203, test_Accuracy: 0.8831
Epoch: [ 0] [  154/  468] time: 36.5801, train_loss: 0.53012407, train_accuracy: 0.8594, test_Accuracy: 0.8839
Epoch: [ 0] [  155/  468] time: 36.8723, train_loss: 0.54436064, train_accuracy: 0.8438, test_Accuracy: 0.8849
Epoch: [ 0] [  156/  468] time: 37.1316, train_loss: 0.42238948, train_accuracy: 0.9141, test_Accuracy: 0.8858
Epoch: [ 0] [  157/  468] time: 37.4199, train_loss: 0.51981306, train_accuracy: 0.8594, test_Accuracy: 0.8859
E

Epoch: [ 0] [  223/  468] time: 52.6640, train_loss: 0.34083259, train_accuracy: 0.9219, test_Accuracy: 0.9022
Epoch: [ 0] [  224/  468] time: 52.8865, train_loss: 0.37403452, train_accuracy: 0.8984, test_Accuracy: 0.9033
Epoch: [ 0] [  225/  468] time: 53.0999, train_loss: 0.36038214, train_accuracy: 0.8984, test_Accuracy: 0.9034
Epoch: [ 0] [  226/  468] time: 53.3353, train_loss: 0.39114222, train_accuracy: 0.8828, test_Accuracy: 0.9045
Epoch: [ 0] [  227/  468] time: 53.5846, train_loss: 0.37096137, train_accuracy: 0.9219, test_Accuracy: 0.9042
Epoch: [ 0] [  228/  468] time: 53.8200, train_loss: 0.21799879, train_accuracy: 0.9375, test_Accuracy: 0.9041
Epoch: [ 0] [  229/  468] time: 54.0394, train_loss: 0.38300905, train_accuracy: 0.9141, test_Accuracy: 0.9029
Epoch: [ 0] [  230/  468] time: 54.3156, train_loss: 0.39885432, train_accuracy: 0.8984, test_Accuracy: 0.9030
Epoch: [ 0] [  231/  468] time: 54.6088, train_loss: 0.34739822, train_accuracy: 0.9297, test_Accuracy: 0.9035
E

Epoch: [ 0] [  297/  468] time: 71.3181, train_loss: 0.29378834, train_accuracy: 0.9297, test_Accuracy: 0.9119
Epoch: [ 0] [  298/  468] time: 71.5495, train_loss: 0.26722875, train_accuracy: 0.9375, test_Accuracy: 0.9123
Epoch: [ 0] [  299/  468] time: 71.7679, train_loss: 0.40422827, train_accuracy: 0.8750, test_Accuracy: 0.9123
Epoch: [ 0] [  300/  468] time: 72.0282, train_loss: 0.48552161, train_accuracy: 0.8672, test_Accuracy: 0.9123
Epoch: [ 0] [  301/  468] time: 72.2427, train_loss: 0.50382727, train_accuracy: 0.8438, test_Accuracy: 0.9138
Epoch: [ 0] [  302/  468] time: 72.4671, train_loss: 0.39696655, train_accuracy: 0.8906, test_Accuracy: 0.9123
Epoch: [ 0] [  303/  468] time: 72.7541, train_loss: 0.34829238, train_accuracy: 0.9219, test_Accuracy: 0.9113
Epoch: [ 0] [  304/  468] time: 72.9705, train_loss: 0.33662352, train_accuracy: 0.9062, test_Accuracy: 0.9115
Epoch: [ 0] [  305/  468] time: 73.1719, train_loss: 0.29154527, train_accuracy: 0.9141, test_Accuracy: 0.9106
E

Epoch: [ 0] [  371/  468] time: 88.7233, train_loss: 0.30346075, train_accuracy: 0.8906, test_Accuracy: 0.9155
Epoch: [ 0] [  372/  468] time: 88.9697, train_loss: 0.22613958, train_accuracy: 0.9297, test_Accuracy: 0.9161
Epoch: [ 0] [  373/  468] time: 89.2709, train_loss: 0.34397528, train_accuracy: 0.9219, test_Accuracy: 0.9169
Epoch: [ 0] [  374/  468] time: 89.4903, train_loss: 0.23174119, train_accuracy: 0.9297, test_Accuracy: 0.9176
Epoch: [ 0] [  375/  468] time: 89.6917, train_loss: 0.31540108, train_accuracy: 0.9141, test_Accuracy: 0.9175
Epoch: [ 0] [  376/  468] time: 89.8742, train_loss: 0.34807038, train_accuracy: 0.8906, test_Accuracy: 0.9178
Epoch: [ 0] [  377/  468] time: 90.0608, train_loss: 0.24630126, train_accuracy: 0.9375, test_Accuracy: 0.9176
Epoch: [ 0] [  378/  468] time: 90.2562, train_loss: 0.24458089, train_accuracy: 0.9219, test_Accuracy: 0.9181
Epoch: [ 0] [  379/  468] time: 90.4547, train_loss: 0.32412627, train_accuracy: 0.9141, test_Accuracy: 0.9184
E

Epoch: [ 0] [  445/  468] time: 104.6748, train_loss: 0.17607862, train_accuracy: 0.9531, test_Accuracy: 0.9221
Epoch: [ 0] [  446/  468] time: 104.9122, train_loss: 0.32634285, train_accuracy: 0.9141, test_Accuracy: 0.9228
Epoch: [ 0] [  447/  468] time: 105.2323, train_loss: 0.23183441, train_accuracy: 0.9531, test_Accuracy: 0.9229
Epoch: [ 0] [  448/  468] time: 105.5066, train_loss: 0.39718884, train_accuracy: 0.9062, test_Accuracy: 0.9228
Epoch: [ 0] [  449/  468] time: 105.7051, train_loss: 0.23451421, train_accuracy: 0.9297, test_Accuracy: 0.9225
Epoch: [ 0] [  450/  468] time: 105.9205, train_loss: 0.15730217, train_accuracy: 0.9609, test_Accuracy: 0.9229
Epoch: [ 0] [  451/  468] time: 106.1229, train_loss: 0.23028801, train_accuracy: 0.9531, test_Accuracy: 0.9230
Epoch: [ 0] [  452/  468] time: 106.4012, train_loss: 0.23141122, train_accuracy: 0.9375, test_Accuracy: 0.9227
Epoch: [ 0] [  453/  468] time: 106.6286, train_loss: 0.24127492, train_accuracy: 0.9453, test_Accuracy:

In [ ]:
Test accuracy =92.61%가 나왔다.